In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import numpy as np
import pandas as pd
import xarray as xr

In [4]:
xr.set_options(keep_attrs=True)

In [5]:
import ecephys_analyses as ea
import ecephys.plot as eplt

In [6]:
from scipy.stats import mode

def estimate_fs(da):
    sample_period = mode(np.diff(da.time.values)).mode[0]
    assert isinstance(sample_period, np.timedelta64)
    sample_period = sample_period / pd.to_timedelta(1, 's')
    return 1 / sample_period

In [7]:
from ripple_detection.core import gaussian_smooth

def get_smoothed_da(da, smoothing_sigma=10, in_place=False):
    if not in_place:
        da = da.copy()
    da.values = gaussian_smooth(da, smoothing_sigma, estimate_fs(da))
    return da    

def get_smoothed_ds(ds, smoothing_sigma=10, in_place=False):
    if not in_place:
        ds = ds.copy()
    for da_name, da in ds.items():
        ds[da_name] = get_smoothed_da(da, smoothing_sigma, in_place)
    return ds

In [8]:
from ecephys.scoring import get_timepoint_states

def filter_states(ds, hypnogram, states):
    timepoint_states = get_timepoint_states(hypnogram, ds.time)
    return ds.where(np.isin(timepoint_states, states))

In [63]:
def load_condition_data(subject, experiment, condition, power_as_percentage_of=None):
    hypnogram = ea.load_hypnogram(subject, experiment, condition)    
    spws = ea.load_spws(subject, experiment, condition, condition_start_dt=hypnogram.start_time.min())
    bp = ea.load_bandpower(subject, experiment, condition)
    if power_as_percentage_of:
        bp = bp / power_as_percentage_of * 100
    
    bp = get_smoothed_ds(bp, smoothing_sigma=10, in_place=True)
    
    return hypnogram, spws, bp

In [102]:
def plot_condition(hypnogram, spws, power):

    fig, (mpta_swa_ax, sr_swa_ax, density_ax, duration_ax, amplitude_ax, integral_ax) = plt.subplots(6, 1, figsize=(22,12))
    xmargin=0
    
    mpta_swa_ylabel = 'CTX delta, WM ref'
    sr_swa_ylabel = 'SR delta, WM ref'
    mpta_swa_ylabel = mpta_swa_ylabel + ', \n % BSL NREM mean'
    sr_swa_ylabel = sr_swa_ylabel + ', \n % BSL NREM mean'

    # units = (uV^2/Hz)
    sns.lineplot(x=power.time.values, y=power.mpta_delta_wm_ref.values, color='black', ax=mpta_swa_ax)
    mpta_swa_ax.set(xlabel=None, ylabel=mpta_swa_ylabel, ylim=MPTA_SWA_LIM, xmargin=xmargin, xticks=[])

    sns.lineplot(x=power.time.values, y=power.sr_delta_wm_ref.values, color='black', ax=sr_swa_ax)
    sr_swa_ax.set(xlabel=None, ylabel=sr_swa_ylabel, ylim=SR_SWA_LIM, xmargin=xmargin, xticks=[])
    
    mpta_swa_ax.axhline(100, color='r', linestyle=':')
    sr_swa_ax.axhline(100, color='r', linestyle=':')

    binwidth = 10
    nbins = np.int((spws.start_time.max() - spws.start_time.min()).total_seconds() / binwidth)
    sns.histplot(data=spws, x='start_time', bins=nbins, color='black', ax=density_ax)
    density_ax.set(xlabel=None, ylabel='Frequency (Hz)', ylim=DENSITY_LIM, xmargin=xmargin, xticks=[])

    sns.scatterplot(data=spws, x="start_time", y="duration", s=2, ax=duration_ax)
    duration_ax.set(xlabel=None, ylabel="Duration (s)", ylim=DURATION_LIM, xmargin=xmargin, xticks=[])

    sns.scatterplot(data=spws, x='start_time', y='sink_amplitude', s=2, ax=amplitude_ax)
    amplitude_ax.set(xlabel=None, ylabel="Sink amplitude (mA/mm)", ylim=AMPLITUDE_LIM, xmargin=xmargin, xticks=[])

    sns.scatterplot(data=spws, x='start_time', y='sink_integral', s=2, ax=integral_ax)
    integral_ax.set(xlabel="Event time (month-date hour)", ylabel="Sink integral (mA * ms)", ylim=INTEGRAL_LIM, xmargin=xmargin)

    for ax in (mpta_swa_ax, sr_swa_ax, density_ax, duration_ax, amplitude_ax, integral_ax):
        eplt.plot_hypnogram_overlay(hypnogram, ax=ax)

    plt.tight_layout()

In [65]:
subject = "Allan"
experiment = "atropine"
condition = "all"

In [69]:
subject = "Allan"
experiment = "dexmedetomidine"
condition = "all"

In [57]:
subject = "Doppio"
experiment = "atropine-10mg"
condition = "all"

In [54]:
subject = "Doppio"
experiment = "dexmedetomidine"
condition = "all"

In [103]:
subject = "Valentino"
experiment = "atropine"
condition = "all"

In [94]:
subject = "Valentino"
experiment = "sevo-dex"
condition = "all"
state_colors = {'None': 'white', 'SevoDex': 'plum', 'Dex': 'thistle', 'Wake': 'palegreen', 'Art': 'crimson'}

In [104]:
MPTA_SWA_LIM = SR_SWA_LIM = DENSITY_LIM = DURATION_LIM = AMPLITUDE_LIM = INTEGRAL_LIM = (None, None)

In [105]:
(hypnogram, spws, power) = load_condition_data(subject, experiment, condition)

In [106]:
plot_condition(hypnogram, spws, power)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …